# Transformer



传统的RNN中，使用word2vec做语句编码，它很明显的一个问题是，不同语境中，同一个单词的编码是完全相同的。**Transformer**解决了这个问题。下图是Transformer的整体架构：

![overview](./images/overview-architechture.png)

## Encoder

Transformer的核心思想是**Attention**, 即找到一句话中的关注点，或者说比较重要的一些单词。

### Self-Attention

通过Self-Attention编码来计算每个单词的重要性。大概思想是，把句子中的每一个单词，分别与句子中每个单词(包含自身)计算相关性，并进行权重求和，从而得到一个单词的新特性（与输入维度相同）,这个新特性会包含整句全局环境。

#### 如何计算

![self-attention](./images/self-attention.png)

每个单词都尝试构建三个向量（一句话就是三个矩阵），query: 查询，key: 查询的答案，value: 自身的特征。那么q,k,v如何计算？

![self-attention-weights](./images/self-attention-weights.png)

初始时，需要初始化三个权重矩阵，分别计算出q,k,v矩阵。如何更新权重？

![self-attention-qk](./images/self-attention-qk.png)

![self-attention-formula](./images/self-attention-formula.png)

通过上图，得到了一句话中，每个单词的新特征，最后得到的是一个z向量，与原始input同维。而上述**三个权重矩阵的更新**，则是通过最终decoder的输出loss反馈来进行的。上图中dk是维度，因为不同的词，得到的embedding(输入)长度是不同的，这会导致长的词，权重变大的可能增大，因此做一个平均的感觉。

下图总结了Attention计算的整体流程。

![attention-overview](./images/attention-overview.png)

### Multi-headed(多头机制，多头特征)

通俗理解，同一个事物，从不同角度，就会展现不同的特征。多头机制就是对一组样本同时采集多种特征的方式。对应到上文的qkv,那么同一个词，会每次计算出多组qkv,在求z时(参考上文)，**每两个词之间的对应组qkv进行计算**, 那么，显然同一个关系我们会得到多个z，这时候需要把多个z进行拼接，从而得到最终的z。注意，多头中的qkvwz长度会较短（实际计算不是平均长度）,**最终拼接出的z长度与不使用多头长度相同**。

### 堆叠多层
简单来说self-attention的计算需要进行多次，但是每次计算方法都是相同的。常见的是8，12，24层，理论上层数越多越好。

![attention-multi-layer](./images/attention-multiple-layer.png)

### 位置信息表达
根据上面的编码方式，我们发现，类似：‘我吃鱼’与’鱼吃我‘，得到的编码是相同的，这意味着编码不具备位置信息。于是，为了添加位置信息，transformer增加了positional encoding.下图中的’+‘运算是指向量对应相加，比如：x1:1.7,0.7,1.5 + t1:0,0,1 = 1.7,0.7,2.5。位置编码一般不需要权重更新，不过也有比较水的论文更新做创新。。。所以位置编码在attention计算之前。
![pos-encoding](./images/pos-encoding.png)


### Add与Normalize
首先Normalize可以参考[batch_norm](../tensorflow/batch_norm.ipynb)

Add其实与[Resnet](../tensorflow/cnn/Resnet_Densenet.ipynb)的思路一致，其实就是基本的残差链接方式

![add-normalize](./images/add-normalize.png)

### 如何使用Encoder得到的特征进行分类

这里以BERT类比，只需要在一句话中，额外添加一个[CLS]向量来与词分别做attention，从而统计出整句特征。然后就可以通过CLS向量去做下一步的分类了。
![categary-transformer](./images/categary-transformer.png)

## Decoder
decoder主要包含两个新东西，以机器翻译为例。

首先是attention的计算，attention包含self-attention和cross-attention两步，self-attention是在已经翻译出来的词(机器翻译是一个词一个词进行翻译的)之间进行计算；cross-attention是将翻译出来的每一个词与encoder的每一个词分别进行attention计算。翻译出的词如何初始化？随机一个词，然后开始优化。self-attention和cross-attention的顺序不重要，都可以，结合具体算法调试。

第二是MASK机制，mask机制是为了控制当前词可以和哪些词计算关系。比如'我吃苹果'翻译'I eat apple', I: 100, eat: 110, apple: 111。其中1表示可以参与attention计算，0不参与。

![decoder](./images/decoder.png)

## BERT(Bidirectional Encoder Representations from Transformers)
BERT是一种基于Transformer模型的预训练语言表示方法，本质就是transformer的encoder部分，并不需要标签，有语料就可以训练。

![bert](./images/bert.png)

### 如何训练bert

第一种方法是Google使用的方法，encoder之后，通过语料库中词的可能性来表示多分类。
![bert-train1](./images/bert-train1.png)

第二种方法，暂时不了解
![bert-train2](./images/bert-train2.png)

### 如何使用bert

![bert-use](./images/bert-use.png)

![bert-use2](./images/bert-use2.png)